In [1]:

from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:

#Importing libraries
import pandas as pd
import os
import numpy as np
#import nltk
os.chdir("/content/drive/My Drive/Innoplexus/")
#import random
random.seed(123)
#import re

#Loading dataset
train = pd.read_csv('train_POS.csv')
train["tag"]=train["tag"].replace({"B-indications":"I-indications"})
#test = pd.read_csv('test_POS.csv')

#train.head()



In [0]:
#train["tag"].unique()

In [3]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("Sent_ID").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

getter = SentenceGetter(train)
#sent = getter.get_next()
sentences = getter.sentences

print(sentences[0])

words = list(set(train["Word"].values))
words.append("ENDPAD")
n_words = len(words)

tags = list(set(train["tag"].values))
n_tags = len(tags)

n_tags

'''lengthlist=[]
for i in sentences:
  lengthlist.append(len(i))
  '''

#max(lengthlist)
#lengthlist.sort()
#print(lengthlist)
#print(len(lengthlist))



'''count = 0 
for i in lengthlist:
  if i>70:
    count+=1
    
print(count*100/len(lengthlist))
'''



[('Obesity', 'NN', 'O'), ('in', 'IN', 'O'), ('Low-', 'JJ', 'O'), ('and', 'CC', 'O'), ('Middle-Income', 'JJ', 'O'), ('Countries', 'NNS', 'O'), (':', ':', 'O'), ('Burden', 'NN', 'O'), (',', ',', 'O'), ('Drivers', 'NNS', 'O'), (',', ',', 'O'), ('and', 'CC', 'O'), ('Emerging', 'VBG', 'O'), ('Challenges', 'NNS', 'O'), ('.', '.', 'O')]


'count = 0 \nfor i in lengthlist:\n  if i>70:\n    count+=1\n    \nprint(count*100/len(lengthlist))\n'

In [0]:
max_len = 50
tag2idx = {t: i for i, t in enumerate(tags)}

X = [[w[0] for w in s] for s in sentences]

new_X = []
for seq in X:
    new_seq = []
    for i in range(max_len):
        try:
            new_seq.append(seq[i])
        except:
            new_seq.append("__PAD__")
    new_X.append(new_seq)
X = new_X

y = [[tag2idx[w[2]] for w in s] for s in sentences]


In [5]:

from keras.preprocessing.sequence import pad_sequences
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])

from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1, random_state=2019)

batch_size = 32
import tensorflow as tf
import tensorflow_hub as hub
from keras import backend as K

sess = tf.Session()
K.set_session(sess)

elmo_model = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

def ElmoEmbedding(x):
    return elmo_model(inputs={
                            "tokens": tf.squeeze(tf.cast(x, tf.string)),
                            "sequence_len": tf.constant(batch_size*[max_len])
                      },
                      signature="tokens",
                      as_dict=True)["elmo"]



Using TensorFlow backend.
W0324 12:49:20.310446 139915035600768 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


Instructions for updating:
Colocations handled automatically by placer.


W0324 12:49:27.572036 139915035600768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [6]:
from keras.models import Model, Input
from keras.layers.merge import add
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Lambda

input_text = Input(shape=(max_len,), dtype=tf.string)
embedding = Lambda(ElmoEmbedding, output_shape=(max_len, 1024))(input_text)
x = Bidirectional(LSTM(units=512, return_sequences=True,
                       recurrent_dropout=0.2, dropout=0.2))(embedding)

x_rnn = Bidirectional(LSTM(units=512, return_sequences=True,
                           recurrent_dropout=0.2, dropout=0.2))(x)
x = add([x, x_rnn])  # residual connection to the first biLSTM
out = TimeDistributed(Dense(n_tags, activation="softmax"))(x)

model = Model(input_text, out)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy")
#, metrics=["accuracy"]


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0324 12:49:28.790217 139915035600768 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


W0324 12:49:30.210091 139915035600768 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:

X_tr, X_val = X_tr[:5000*batch_size], X_tr[-379*batch_size:]
y_tr, y_val = y_tr[:5000*batch_size], y_tr[-379*batch_size:]
y_tr = y_tr.reshape(y_tr.shape[0], y_tr.shape[1], 1)
y_val = y_val.reshape(y_val.shape[0], y_val.shape[1], 1)

# 5000 we can change



In [8]:

del train
del sentences
import gc
#gc.collect()


0

In [9]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()



[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16312344093491430389
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 15902001402004427099
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 2512100414812787756
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11276946637
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2251498828991572568
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


['/job:localhost/replica:0/task:0/device:GPU:0']

In [10]:
history = model.fit(np.array(X_tr), y_tr, validation_data=(np.array(X_val), y_val),
                    batch_size=batch_size,
                    epochs=3, verbose=1)


Instructions for updating:
Use tf.cast instead.


W0324 12:50:30.777068 139915035600768 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Train on 160000 samples, validate on 12128 samples
Epoch 1/3
 33056/160000 [=====>........................] - ETA: 1:11:57 - loss: 0.0232

KeyboardInterrupt: ignored

In [0]:
i = 54
p = model.predict(np.array(X_te[i:i+batch_size]))[0]
p = np.argmax(p, axis=-1)
print("{:15} {:5}: ({})".format("Word", "Pred", "True"))
print("="*30)
for w, true, pred in zip(X_te[i], y_te[i], p):
    if w != "__PAD__":
        print("{:15}:{:5} ({})".format(w, tags[pred], tags[true]))
